# atmodeller

## Tutorial 3: Monte Carlo experiment

We can devise a simple Monte Carlo approach to sample the probable atmospheres that can arise for different planetary conditions.

Import the required packages and set the package logger to the INFO level. For more output you could instead set it to DEBUG.

In [1]:
from atmodeller import InteriorAtmosphereSystem, Planet, SystemConstraint, logger, MassConstraint, BufferedFugacityConstraint
from atmodeller.thermodynamics import NoSolubility,  ChemicalComponent, GasSpecies
from atmodeller.solubilities import PeridotiteH2O, BasaltDixonCO2, BasaltLibourelN2
from atmodeller.utilities import earth_oceans_to_kg
import numpy as np
import csv
import logging

logger.setLevel(logging.INFO)

In [2]:
species: list[ChemicalComponent] = []
species.append(GasSpecies(chemical_formula='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='N2', solubility=BasaltLibourelN2(), solid_melt_distribution_coefficient=0))
species

09:51:20 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: H2O (H2O)
09:51:20 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: H2 (H2)
09:51:20 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: O2 (O2)
09:51:20 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: CO (CO)
09:51:20 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: CO2 (CO2)
09:51:20 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: N2 (N2)


[GasSpecies(chemical_formula='H2O', common_name='H2O', formula=Formula('H2O'), solubility=<atmodeller.solubilities.PeridotiteH2O object at 0x126fcedd0>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='H2', common_name='H2', formula=Formula('H2'), solubility=<atmodeller.solubilities.NoSolubility object at 0x11cfdaa90>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='O2', common_name='O2', formula=Formula('O2'), solubility=<atmodeller.solubilities.NoSolubility object at 0x126fcd350>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='CO', common_name='CO', formula=Formula('CO'), solubility=<atmodeller.solubilities.NoSolubility object at 0x126fce650>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='CO2', common_name='CO2', formula=Formula('CO2'), solubility=<atmodeller.solubilities.BasaltDixonCO2 object at 0x126fcc250>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='N2', common_name

In [3]:
planet: Planet = Planet()
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

09:51:23 - atmodeller.thermodynamics      - INFO      - Creating a new planet
09:51:23 - atmodeller.thermodynamics      - INFO      - Mantle mass (kg) = 4208261222595110885130240.000000
09:51:23 - atmodeller.thermodynamics      - INFO      - Mantle melt fraction = 1.000000
09:51:23 - atmodeller.thermodynamics      - INFO      - Core mass fraction = 0.295335
09:51:23 - atmodeller.thermodynamics      - INFO      - Planetary radius (m) = 6371000.000000
09:51:23 - atmodeller.thermodynamics      - INFO      - Planetary mass (kg) = 5972000000000000327155712.000000
09:51:23 - atmodeller.thermodynamics      - INFO      - Surface temperature (K) = 2000.000000
09:51:23 - atmodeller.thermodynamics      - INFO      - Surface gravity (m/s^2) = 9.819973
09:51:23 - atmodeller.thermodynamics      - INFO      - Melt Composition = None
09:51:23 - atmodeller.core                - INFO      - Creating a new interior-atmosphere system
09:51:23 - atmodeller.core                - INFO      - Species = ['CO',

In [4]:
number_of_realisations: int = 2

# Parameters are normally distributed between bounds.
number_ocean_moles: np.ndarray = np.random.uniform(1, 10, number_of_realisations)
ch_ratios: np.ndarray = np.random.uniform(0.1, 1, number_of_realisations)
fo2_shifts: np.ndarray = np.random.uniform(-4, 4, number_of_realisations)

# Store the output in a list.
out: list[dict[str, float]] = []

# ppmw of Nitrogen in the mantle. 2.8 is the mantle value of N.
N_ppmw: float = 2.8

# The nitrogen mass is constant.
mass_N: float = N_ppmw * 1.0e-6 * planet.mantle_mass

for realisation in range(number_of_realisations):

    mass_H: float = earth_oceans_to_kg(number_ocean_moles[realisation])
    mass_C: float = ch_ratios[realisation] * mass_H
    constraints: list[SystemConstraint] = [
        MassConstraint(species="H", value=mass_H),
        MassConstraint(species="C", value=mass_C),
        MassConstraint(species="N", value=mass_N),
        BufferedFugacityConstraint(log10_shift=fo2_shifts[realisation])
    ]
    # Recall that changing attributes on the planet 'object' will be 'seen' by interior_atmosphere.
    out_realisation = interior_atmosphere.solve(constraints)

    # Include the parameters in the output.
    out_realisation['number_ocean_moles'] = number_ocean_moles[realisation]
    out_realisation['ch_ratio'] = ch_ratios[realisation]
    out_realisation['fo2_shift'] = fo2_shifts[realisation]

    out.append(out_realisation)

    filename: str = "atmodeller_monte_carlo_tutorial3.csv"
    print("Writing output to: %s", filename)
    fieldnames: list[str] = list(out[0].keys())
    with open(filename, "w", newline="", encoding="utf-8") as csvfile:
        writer: csv.DictWriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(out)

21:52:13 - atmodeller.core                - INFO      - Constraints: [MassConstraint(species='H', value=1.864331549160412e+20, field='mass'),
 MassConstraint(species='C', value=1.4026609110004897e+20, field='mass'),
 MassConstraint(species='N', value=1.178313142326631e+19, field='mass'),
 BufferedFugacityConstraint(species='O2',
                            fugacity=<atmodeller.thermodynamics.IronWustiteBufferHirschmann object at 0x1271878d0>,
                            log10_shift=2.204603803580289,
                            field='fugacity')]
21:52:13 - atmodeller.core                - INFO      - Mixed pressure and mass constraints so attempting to solve a non-linear system of equations
21:52:13 - atmodeller.core                - INFO      - 3 additional (not fugacity) constraint(s) are necessary to solve the system
21:52:13 - atmodeller.core                - INFO      - Row 00: Reaction 0: 1.0 CO + 0.5 O2 = 1.0 CO2
21:52:14 - atmodeller.core                - INFO      - Row 01: R

Writing output to: %s atmodeller_monte_carlo_tutorial3.csv


21:52:18 - atmodeller.core                - INFO      - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
21:52:18 - atmodeller.core                - INFO      - Row 02: Setting O2 partial pressure
21:52:18 - atmodeller.core                - INFO      - 3 additional (not fugacity) constraint(s) are necessary to solve the system
21:52:18 - atmodeller.core                - INFO      - Row 00: Reaction 0: 1.0 CO + 0.5 O2 = 1.0 CO2
21:52:18 - atmodeller.core                - INFO      - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
21:52:18 - atmodeller.core                - INFO      - Row 02: Setting O2 partial pressure
21:52:18 - atmodeller.core                - INFO      - 3 additional (not fugacity) constraint(s) are necessary to solve the system
21:52:18 - atmodeller.core                - INFO      - Row 00: Reaction 0: 1.0 CO + 0.5 O2 = 1.0 CO2
21:52:18 - atmodeller.core                - INFO      - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
21:52:19 - atmodeller.core                

Writing output to: %s atmodeller_monte_carlo_tutorial3.csv


The simulation data is output as:

In [5]:
out

[{'CO': 18.49054830925427,
  'H2': 0.04407328099707975,
  'N2': 2.967259416121338,
  'O2': 1.3935573818265822e-05,
  'CO2': 52.44091378256771,
  'H2O': 0.5698976214376456,
  'number_ocean_moles': 1.202842282523385,
  'ch_ratio': 0.7523666654850999,
  'fo2_shift': 2.204603803580289},
 {'CO': 10.878662312345321,
  'H2': 0.1579805395379308,
  'N2': 2.6370808389972424,
  'O2': 1.8330685056599262e-06,
  'CO2': 11.189814233219227,
  'H2O': 0.7408870000303884,
  'number_ocean_moles': 1.3720201061627009,
  'ch_ratio': 0.2149364762052734,
  'fo2_shift': 1.3236576432044975}]